# 32bit AES

Supported setups:

SCOPES:

* OPENADC
* CWNANO

PLATFORMS:

* CWLITEARM
* CWNANO

Previous AES tutorials (even on 32-bit targets) ran 8-bit modes of operation. We can target typical implementation on Arm devices which actually looks a little different.

This tutorial is ONLY possible if you have an ARM target. For example the CWLite Arm target or the UFO Board with an STM32F target (or similar).

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
N = 5000
CHECK_CORR = False

In [10]:
##Debugging usb error
import chipwhisperer as cw
cw.__package__

'chipwhisperer'

## Background

A 32-bit machine can operate on 32-bit words, so it seems wasteful to use the same 8-bit operations. Indeed we can speed up the AES operation considerably by generating several tables (called T-Tables), as was described in the book [The Design of Rijndael](http://www.springer.com/gp/book/9783540425809) which was published by the authors of AES.

In order to take advantage of our 32 bit machine, we can examine a typical round of AES. With the exception of the final round, each round looks like:

$\text{a = Round Input}$

$\text{b = SubBytes(a)}$

$\text{c = ShiftRows(b)}$

$\text{d = MixColumns(c)}$

$\text{a' = AddRoundKey(d) = Round Output}$

We'll leave AddRoundKey the way it is. The other operations are:

$b_{i,j} = \text{sbox}[a_{i,j}]$

$\left[ \begin{array} { c } { c _ { 0 , j } } \\ { c _ { 1 , j } } \\ { c _ { 2 , j } } \\ { c _ { 3 , j } } \end{array} \right] = \left[ \begin{array} { l } { b _ { 0 , j + 0 } } \\ { b _ { 1 , j + 1 } } \\ { b _ { 2 , j + 2 } } \\ { b _ { 3 , j + 3 } } \end{array} \right]$

$\left[ \begin{array} { l } { d _ { 0 , j } } \\ { d _ { 1 , j } } \\ { d _ { 2 , j } } \\ { d _ { 3 , j } } \end{array} \right] = \left[ \begin{array} { l l l l } { 02 } & { 03 } & { 01 } & { 01 } \\ { 01 } & { 02 } & { 03 } & { 01 } \\ { 01 } & { 01 } & { 02 } & { 03 } \\ { 03 } & { 01 } & { 01 } & { 02 } \end{array} \right] \times \left[ \begin{array} { c } { c _ { 0 , j } } \\ { c _ { 1 , j } } \\ { c _ { 2 , j } } \\ { c _ { 3 , j } } \end{array} \right]$

Note that the ShiftRows operation $b_{i, j+c}$ is a cyclic shift and the matrix multiplcation in MixColumns denotes the xtime operation in GF($2^8$).

It's possible to combine all three of these operations into a single line. We can write 4 bytes of $d$ as the linear combination of four different 4 byte vectors:

$\left[ \begin{array} { l } { d _ { 0 , j } } \\ { d _ { 1 , j } } \\ { d _ { 2 , j } } \\ { d _ { 3 , j } } \end{array} \right] = \left[ \begin{array} { l } { 02 } \\ { 01 } \\ { 01 } \\ { 03 } \end{array} \right] \operatorname { sbox } \left[ a _ { 0 , j + 0 } \right] \oplus \left[ \begin{array} { l } { 03 } \\ { 02 } \\ { 01 } \\ { 01 } \end{array} \right] \operatorname { sbox } \left[ a _ { 1 , j + 1 } \right] \oplus \left[ \begin{array} { c } { 01 } \\ { 03 } \\ { 02 } \\ { 01 } \end{array} \right] \operatorname { sbox } \left[ a _ { 2 , j + 2 } \right] \oplus \left[ \begin{array} { c } { 01 } \\ { 01 } \\ { 03 } \\ { 02 } \end{array} \right] \operatorname { sbox } \left[ a _ { 3 , j + 3 } \right]$

Now, for each of these four components, we can tabulate the outputs for every possible 8-bit input:

$T _ { 0 } [ a ] = \left[ \begin{array} { l l } { 02 \times \operatorname { sbox } [ a ] } \\ { 01 \times \operatorname { sbox } [ a ] } \\ { 01 \times \operatorname { sbox } [ a ] } \\ { 03 \times \operatorname { sbox } [ a ] } \end{array} \right]$

$T _ { 1 } [ a ] = \left[ \begin{array} { l } { 03 \times \operatorname { sbox } [ a ] } \\ { 02 \times \operatorname { sbox } [ a ] } \\ { 01 \times \operatorname { sbox } [ a ] } \\ { 01 \times \operatorname { sbox } [ a ] } \end{array} \right]$

$T _ { 2 } [ a ] = \left[ \begin{array} { l l } { 01 \times \operatorname { sbox } [ a ] } \\ { 03 \times \operatorname { sbox } [ a ] } \\ { 02 \times \operatorname { sbox } [ a ] } \\ { 01 \times \operatorname { sbox } [ a ] } \end{array} \right]$

$T _ { 3 } [ a ] = \left[ \begin{array} { l l } { 01 \times \operatorname { sbox } [ a ] } \\ { 01 \times \operatorname { sbox } [ a ] } \\ { 03 \times \operatorname { sbox } [ a ] } \\ { 02 \times \operatorname { sbox } [ a ] } \end{array} \right]$

These tables have 2^8 different 32-bit entries, so together the tables take up 4 kB. Finally, we can quickly compute one round of AES by calculating

$\left[ \begin{array} { l } { d _ { 0 , j } } \\ { d _ { 1 , j } } \\ { d _ { 2 , j } } \\ { d _ { 3 , j } } \end{array} \right] = T _ { 0 } \left[ a _ { 0 } , j + 0 \right] \oplus T _ { 1 } \left[ a _ { 1 } , j + 1 \right] \oplus T _ { 2 } \left[ a _ { 2 } , j + 2 \right] \oplus T _ { 3 } \left[ a _ { 3 } , j + 3 \right]$

All together, with AddRoundKey at the end, a single round now takes 16 table lookups and 16 32-bit XOR operations. This arrangement is much more efficient than the traditional 8-bit implementation. There are a few more tradeoffs that can be made: for instance, the tables only differ by 8-bit shifts, so it's also possible to store only 1 kB of lookup tables at the expense of a few rotate operations.

Note that T-tables don't have a big effect on AES from a side-channel analysis perspective. The SubBytes output is still buried in the T-tables and the other operations are linear, so it's still possible to attack 32-bit AES using the same 8-bit attack methods.

## Firmware

Firmware is the same as previous AES examples, except this time we'll need to build it using MBEDTLS (previous examples used TINYAES128C):

In [2]:
%%bash
cd ../hardware/victims/firmware/
mkdir -p simpleserial-aes-captureTrace_manoj_f303 && cp -r simpleserial-aes/* $_

In [3]:
CRYPTO_TARGET = "MBEDTLS"

In [5]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-aes-captureTrace_manoj_f303
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- simpleserial-aes-CWLITEARM.hex
rm -f -- simpleserial-aes-CWLITEARM.eep
rm -f -- simpleserial-aes-CWLITEARM.cof
rm -f -- simpleserial-aes-CWLITEARM.elf
rm -f -- simpleserial-aes-CWLITEARM.map
rm -f -- simpleserial-aes-CWLITEARM.sym
rm -f -- simpleserial-aes-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-aes.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s aes-independant.s aes.s
rm -f -- simpleserial-aes.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d aes-independant.d aes.d
rm -f -- simpleserial-aes.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i aes-independant.i aes.i
mkdir .dep
.
-------- begin --------
arm-none-eabi-gcc (15:6.3.1+svn253039-1build1) 6.3.1 20170620
Copyright (C) 2016 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C:

## Running the Attack

Neither the attack, nor the analysis is any different from a normal AES attack as we don't actually care about the T-Table implementation. We will need to capture more traces (500), but our traces also don't have to be as long (only 1500 samples).

### Capturing Traces

In [7]:
%run "Helper_Scripts/Setup_Generic.ipynb"

Traceback (most recent call last):
  File "/home/manoj/Documents/chipwhisperer/software/chipwhisperer/hardware/naeusb/naeusb.py", line 295, in txrx
    response = self.open(serial_number=payload)
  File "/home/manoj/Documents/chipwhisperer/software/chipwhisperer/hardware/naeusb/naeusb.py", line 345, in open
    dev.set_configuration(0)
  File "/home/manoj/.local/lib/python3.6/site-packages/usb/core.py", line 869, in set_configuration
    self._ctx.managed_set_configuration(self, configuration)
  File "/home/manoj/.local/lib/python3.6/site-packages/usb/core.py", line 102, in wrapper
    return f(self, *args, **kwargs)
  File "/home/manoj/.local/lib/python3.6/site-packages/usb/core.py", line 148, in managed_set_configuration
    self.backend.set_configuration(self.handle, cfg.bConfigurationValue)
  File "/home/manoj/.local/lib/python3.6/site-packages/usb/backend/libusb0.py", line 493, in set_configuration
    _check(_lib.usb_set_configuration(dev_handle, config_value))
  File "/home/mano

Warning: Could not connect to "NewAE USB (CWLite/CW1200)". It may have been disconnected, is in an error state, or is being used by another tool.

In [31]:
fw_path = "../hardware/victims/firmware/simpleserial-aes-captureTrace_manoj_dnn/simpleserial-aes-{}.hex".format(PLATFORM)

In [32]:
cw.program_target(scope, prog, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 16699 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 16699 bytes


In [33]:
##Setting preSmaples
scope.adc.samples = 1500 ## Taking 1500 clock cycles since downsampling of 4 is applied
scope.adc.presamples = 0
print("DEcimate= %s" %(scope.adc.decimate))
##Commented for no sampling runs
scope.adc.decimate = 4 ##Downsampling, only 1 in 4 traces will be selected.
## Set timeout to be more than 2sec which is default
scope.adc.timeout = 60
print("adc.timeout= %s" %(scope.adc.timeout))
for i in range(10,15):
    print("%s " %(i))


DEcimate= 1
adc.timeout= 60
10 
11 
12 
13 
14 


In [3]:
import time
for i in range(0,5):
    startTime = time.time()
    print(startTime)
    time.sleep(5)

1586472207.3130867
1586472212.327311
1586472217.3329575
1586472222.3392086
1586472227.3448856


In [37]:
# Capture Traces
import chipwhisperer as cw
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()
##  Changed form 1500 to 5000 for no down sampling runs
scope.adc.samples = 1500
N= 21000 ##Number of samples for complete traingng and testing of DNN


ktp.fixed_key= False ##Ramdomizing the keys
#keyVal = 2
## KeysNotDone for without 'captureTraces_dnn_dir' folder
## Old
## _1
# keysNotDone = [0, 8, 11, 12, 47, 49, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 105, 106, 107, 110, 112, 114, 115, \
#                116, 117, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 132, 133, 134, 136, 137, 139, 141,\
#                142, 172, 174, 175, 176, 177, 178, 180, 182, 183, 185, 186, 187, 188, 189, 190, 192, 193, 194, 196,\
#                197, 198, 200, 203, 204, 205, 209, 211, 213, 216, 217, 219, 220, 222, 223, 224, 225, 226, 228, 230,\
#                232, 234, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254,\
#                255]

## _2
# keysNotDone = [172, 174, 176, 177, 180, 182, 185, 187, 188, 190, 194, 196, 197, 198, 203, 204, 205, 209, 217, 220,\
#                222, 228, 237, 239, 244, 245, 248, 254]
## _3
#keysNotDone = [180, 197, 203, 204, 209, 228, 239, 244, 245, 254]

## _3
#keysNotDone = [204]

##_5
#keysNotDone = [204, 228, 244, 245]

## "dir_1" keys not done
#keysNotDone = [0, 54, 56, 57, 58, 60, 62, 63, 64, 66, 67, 68, 69, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 84, 85, 86, 87, 88,\
#                91, 92, 93, 94, 95, 96, 97, 99, 101, 103, 104, 107, 108, 109, 111, 114, 115, 116, 117, 118, 120, 121, 123, 127,\
#                128, 129, 132, 135, 136, 137, 138, 139, 140, 141, 143, 144, 145, 146, 149, 150, 152, 153, 154, 155, 156, 158,\
#                160, 161, 162, 163, 164, 165, 167, 169, 171, 172, 173, 174, 177, 179, 180, 182, 184, 185, 186, 187, 188, 189,\
#                191, 192, 195, 196, 197, 198, 199, 206, 210, 211, 212, 214, 215, 216, 217, 218, 220, 221, 222, 225, 226, 228,\
#                229, 231, 233, 234, 237, 238, 239, 241, 242, 243, 244, 245, 247, 249, 250, 252, 253]

## These are less than 15000 traces. Keeping a limit of 15000 for tiem sake
#keysNotDone = [163, 167, 172, 173, 177, 185, 186, 188, 196, 198, 210, 220, 225, 228, 234, 237, 239, 247]

#for keyVal in keysNotDone: ##inclusive of left but exclusive of right-side

##dnn_dir_2 1st keys not 21000.
# keysNotDone = [0, 1, 2, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 41, 44, 45, 48, 49, 52, 54, 57, 58, 59, 79,\
#                80, 81, 83, 86, 87, 89, 90, 91, 92, 93, 94, 96, 98, 99, 101, 102, 103, 104, 106, 107, 108, 109,\
#                110, 111, 112,116, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 131, 132, 134, 137, 139,\
#                140, 142, 143, 144, 146, 148,150, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163,\
#                164, 165, 168, 169, 171, 172, 175, 176, 178,179, 180, 181, 184, 185, 186, 187, 194, 195, 196,\
#                197, 199, 200, 204, 205, 207, 208, 209, 211, 215, 216, 218,219, 220, 222, 224, 225, 226, 227,\
#                229, 230, 233, 234, 237, 238, 239, 241, 242, 244, 245, 248, 250, 252]

# keysNotDone = [ 0,124, 125, 126, 127, 129, 131, 132, 134, 137, 139,\
#                140, 142, 143, 144, 146, 148,150, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163,\
#                164, 165, 168, 169, 171, 172, 175, 176, 178,179, 180, 181, 184, 185, 186, 187, 194, 195, 196,\
#                197, 199, 200, 204, 205, 207, 208, 209, 211, 215, 216, 218,219, 220, 222, 224, 225, 226, 227,\
#                229, 230, 233, 234, 237, 238, 239, 241, 242, 244, 245, 248, 250, 252]

keysNotDone = [ 157, 158, 159, 160, 161, 162, 163,\
               164, 165, 168, 169, 171, 172, 175, 176, 178,179, 180, 181, 184, 185, 186, 187, 194, 195, 196,\
               197, 199, 200, 204, 205, 207, 208, 209, 211, 215, 216, 218,219, 220, 222, 224, 225, 226, 227,\
               229, 230, 233, 234, 237, 238, 239, 241, 242, 244, 245, 248, 250, 252]

#for keyVal in keysNotDone:
for keyVal in [1]:
    print("Straing capturing key = %s\n" %(keyVal))
    #prjPath = "projects/captureTraces_dnn_dir/key_" + str(keyVal) + "/part_" + str(count) + ".cwp"
    #prjPath = "projects/captureTraces_dnn_dir_1/key_" + str(keyVal) + "_2.cwp"
    #prjPath = "projects/captureTraces_dnn_dir_2/key_" + str(keyVal) + "_1.cwp"
    prjPath = "projects/captureTraces_dnn_dir_2/key_" + str(keyVal) + ".cwp"
    #For no downsampling run
    #prjPath = "projects/captureTraces_dnn_dir_noDownSampling/key_" + str(keyVal) + ".cwp"
    project = cw.create_project(prjPath, overwrite = True)
    
    ##New caputuring code
    while(1):
        key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
        key[0] = keyVal
        ##print("key= %s\ntext= %s\n\n" %(key, text))
        trace = cw.capture_trace(scope, target, text, key)
        
        ##if trace is None, not saved, goes to the top of while loop
        if trace is None:
            continue
        project.traces.append(trace)
        
        ##if the trace count==21000, break out of the loop and save the project
        if(len(project.traces) == 21000):
            print("Breaking since collected %s traces\n" %(len(project.traces)))
            break
        
    
    #Commenting the below implementation because it would not save 21000 traces
#     for i in tnrange(N, desc='Capturing traces'):
#         key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
#         key[0] = keyVal
#         ##print("key= %s\ntext= %s\n\n" %(key, text))
#         trace = cw.capture_trace(scope, target, text, key)
#         if trace is None:
#             continue
#         project.traces.append(trace)

    startTime = time.time()
    print("Started saving at %s" %(startTime))
    project.save()
    ## Adding 120 sec delay between trace captures
    time.sleep(120)
    print("Done saving %s traces for %s key after %s sec" %(len(project.traces), keyVal, time.time()-startTime))


Straing capturing key = 1

Breaking since collected 21000 traces

Started saving at 1602127130.851358
Done saving 21000 traces for 1 key after 120.79724454879761 sec


In [38]:
## Checcking foor traces less than 21000
lessTraceList = []
## "dir_1" keys not done
# keysNotDone_1 = [0, 54, 56, 57, 58, 60, 62, 63, 64, 66, 67, 68, 69, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 84, 85, 86, 87, 88,\
#                 91, 92, 93, 94, 95, 96, 97, 99, 101, 103, 104, 107, 108, 109, 111, 114, 115, 116, 117, 118, 120, 121, 123, 127,\
#                 128, 129, 132, 135, 136, 137, 138, 139, 140, 141, 143, 144, 145, 146, 149, 150, 152, 153, 154, 155, 156, 158,\
#                 160, 161, 162, 163, 164, 165, 167, 169, 171, 172, 173, 174, 177, 179, 180, 182, 184, 185, 186, 187, 188, 189,\
#                 191, 192, 195, 196, 197, 198, 199, 206, 210, 211, 212, 214, 215, 216, 217, 218, 220, 221, 222, 225, 226, 228,\
#                 229, 231, 233, 234, 237, 238, 239, 241, 242, 243, 244, 245, 247, 249, 250, 252, 253]

# ## These are less than 15000 traces. Keeping a limit of 15000 for tiem sake
# keysNotDone_2 = [163, 167, 172, 173, 177, 185, 186, 188, 196, 198, 210, 220, 225, 228, 234, 237, 239, 247]

## dir_2
keysNotDone_1 = [0, 1, 2, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 41, 44, 45, 48, 49, 52, 54, 57, 58, 59, 79,\
               80, 81, 83, 86, 87, 89, 90, 91, 92, 93, 94, 96, 98, 99, 101, 102, 103, 104, 106, 107, 108, 109,\
               110, 111, 112,116, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 131, 132, 134, 137, 139,\
               140, 142, 143, 144, 146, 148,150, 151, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163,\
               164, 165, 168, 169, 171, 172, 175, 176, 178,179, 180, 181, 184, 185, 186, 187, 194, 195, 196,\
               197, 199, 200, 204, 205, 207, 208, 209, 211, 215, 216, 218,219, 220, 222, 224, 225, 226, 227,\
               229, 230, 233, 234, 237, 238, 239, 241, 242, 244, 245, 248, 250, 252]

for keyVal in range(256):
    prjPath_0 = "projects/captureTraces_dnn_dir_2/key_" + str(keyVal) + ".cwp"
    newProject_0 = cw.open_project(prjPath_0)
    totalLen = len(newProject_0.traces)
    if keyVal in keysNotDone_1:
        prjPath_1 = "projects/captureTraces_dnn_dir_2/key_" + str(keyVal) + "_1.cwp"
        newProject_1 = cw.open_project(prjPath_1)
        totalLen = len(newProject_0.traces) + len(newProject_1.traces)
#     if keyVal in keysNotDone_2:
#         prjPath_2 = "projects/captureTraces_dnn_dir_1/key_" + str(keyVal) + "_2.cwp"
#         newProject_2 = cw.open_project(prjPath_2)
#         totalLen = len(newProject_0.traces) + len(newProject_1.traces) + len(newProject_2.traces)
    
    
    print("key= %s\tnum traces = %s" %(keyVal, totalLen))
    if (totalLen < 21000):
        lessTraceList.append(keyVal)

print (lessTraceList)


key= 0	num traces = 24000
key= 1	num traces = 33000
key= 2	num traces = 24000
key= 3	num traces = 21000
key= 4	num traces = 21000
key= 5	num traces = 21000
key= 6	num traces = 21000
key= 7	num traces = 21000
key= 8	num traces = 21000
key= 9	num traces = 21000
key= 10	num traces = 21000


OSError: File /home/manoj/Documents/chipwhisperer/jupyter/projects/captureTraces_dnn_dir_2/key_11.cwp does not exist or is not a file

In [13]:
## To check the total number of traces saved
lessTraceList = []
for keyVal in range(256):
    prjPath_0 = "projects/captureTraces_dnn_dir_2/key_" + str(keyVal) + ".cwp"
    newProject_0 = cw.open_project(prjPath_0)
    prjPath_1 = "projects/captureTraces_dnn_dir_2/key_" + str(keyVal) + "_1.cwp"
    newProject_1 = cw.open_project(prjPath_1)
#     prjPath_2 = "projects/captureTraces_dnn_dir_1/key_" + str(keyVal) + "_2.cwp"
#     newProject_2 = cw.open_project(prjPath_2)
    
    if ((len(newProject_0.traces)) < 21000):
        print("Number of traces appended= %s for key= %s" %(len(newProject_0.traces), keyVal))
        lessTraceList.append(keyVal)

print(lessTraceList)

Number of traces appended= 12000 for key= 0
Number of traces appended= 12000 for key= 54
Number of traces appended= 12000 for key= 56
Number of traces appended= 12000 for key= 57
Number of traces appended= 12000 for key= 58
Number of traces appended= 12000 for key= 60
Number of traces appended= 12000 for key= 62
Number of traces appended= 12000 for key= 63
Number of traces appended= 3000 for key= 64
Number of traces appended= 3000 for key= 66
Number of traces appended= 12000 for key= 67
Number of traces appended= 3000 for key= 68
Number of traces appended= 3000 for key= 69
Number of traces appended= 12000 for key= 71
Number of traces appended= 3000 for key= 72
Number of traces appended= 3000 for key= 73
Number of traces appended= 12000 for key= 75
Number of traces appended= 3000 for key= 76
Number of traces appended= 3000 for key= 77
Number of traces appended= 3000 for key= 78
Number of traces appended= 12000 for key= 79
Number of traces appended= 3000 for key= 80
Number of traces appe

In [ ]:
## Plot traces
import chipwhisperer as cw
import pandas as pd
import os

## dataFrame to store all the traces
df = pd.DataFrame(columns=['trace', 'key'])
train_df = pd.DataFrame(columns=['trace', 'key'])
dev_df   = pd.DataFrame(columns=['trace', 'key'])
test_df  = pd.DataFrame(columns=['trace', 'key'])

##number of keys to store in one chunk of csv, usually 16 for jupyter
batchSize = 4

count = 0 ##Used as an index for saveFile
wrongCount = 0
lessTraceList = []

keysNotDone_1 = [0, 8, 11, 12, 47, 49, 91, 92, 93, 95, 96, 97, 98, 99, 100, 101, 105, 106, 107, 110, 112, 114, 115, \
               116, 117, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 132, 133, 134, 136, 137, 139, 141,\
               142, 172, 174, 175, 176, 177, 178, 180, 182, 183, 185, 186, 187, 188, 189, 190, 192, 193, 194, 196,\
               197, 198, 200, 203, 204, 205, 209, 211, 213, 216, 217, 219, 220, 222, 223, 224, 225, 226, 228, 230,\
               232, 234, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254,\
               255]
# _2
keysNotDone_2 = [172, 174, 176, 177, 180, 182, 185, 187, 188, 190, 194, 196, 197, 198, 203, 204, 205, 209, 217, 220,\
               222, 228, 237, 239, 244, 245, 248, 254]
# _3
keysNotDone_3 = [180, 197, 203, 204, 209, 228, 239, 244, 245, 254]

## _4
keysNotDone_4 = [204]

##_5
keysNotDone_5 = [204, 228, 244, 245]

## Functions for creating df out of cw files
def oneFileTrace(prjDir, key, fileIndex):
    print("OneFileTrace for key = %s" %(key))
    ## initializing the datafram
    df = pd.DataFrame(columns={'trace', 'key'})
    
    ##Creating prjPath to open the project
    if fileIndex != 0:
        prjPath = prjDir + "/key_" + str(key) + "_" + str(fileIndex) + ".cwp"
    else:
        prjPath = prjDir + "/key_" + str(key) + ".cwp"
    
    new_project = cw.open_project(prjPath)
    
    ## Iterating over each trace and appending it to the df
    for index, trace in enumerate(new_project.traces):
        df = df.append({'trace': trace.wave, 'key': trace.key}, ignore_index=True)
    
    return df

def multiFileTrace(prjDir, key):
    print("multiFileTrace for key = %s" %(key))
    ## initializing the datafram
    df = pd.DataFrame(columns={'trace', 'key'})
    
    ## Setting count to zero for each new key
    countTrace = 0 
    while countTrace <= 5:
        if countTrace  == 0:
            prjPath = prjDir + "/key_" + str(key) + ".cwp"
        else:
            prjPath = prjDir + "/key_" + str(key) + "_" + str(countTrace) + ".cwp"

        if os.path.exists(prjPath):
            print("prjPath= %s\ncountTrace = %s\n" %(prjPath, countTrace))
            new_project = cw.open_project(prjPath)
        else:
            print("%s does not exist" %(prjPath))
            countTrace = countTrace + 1
            continue
        print("project_traces len = %s\nprjPAth = %s" %(len(new_project.traces), prjPath))
        for index, trace in enumerate(new_project.traces):
            df = df.append({'trace': trace.wave, 'key': trace.key}, ignore_index=True)
        
        print("Done appending len= %s for key= %s, count = %s" %(len(df.index),keyVal, count))
        ## increment count
        countTrace = countTrace + 1
    
    print("## Saved all to df in mutliFileTrace")
    return df

def clear_df(df):
    ##Clear contents of the dataFrame
    df = df.drop(df.columns, axis=1)
        
    ## Create new dataFrame
    df = pd.DataFrame(columns=['trace', 'key'])
    print("Cleared df= \n%s" %(df))

## Directory where cw projects are saved
prjDir = "projects/captureTraces_dnn_dir/"

#for keyVal in range(0,6):
for index, keyVal in enumerate([0, 10, 204, 37, 244, 98, 228, 245]):
##for keyVal in keysNotDone_5:
    if keyVal in keysNotDone_5:
        df = multiFileTrace(prjDir, keyVal)
    elif keyVal in keysNotDone_4:
        print("KeyDontDone_4 is already included in 5")
    elif keyVal in keysNotDone_3:
        df = oneFileTrace(prjDir, keyVal, 3)
    elif keyVal in keysNotDone_2:
        df = oneFileTrace(prjDir, keyVal, 2)
    elif keyVal in keysNotDone_1:
        df = oneFileTrace(prjDir, keyVal, 1)
    else:
        df = oneFileTrace(prjDir, keyVal, 0)

#     new_project = cw.open_project(prjPath)
#     #Mprint("project= %s\nprjPath= %s\n" %(new_project.location, prjPath))
# #     print("%s" %(dir(new_project)))
# #     print("%s" %(new_project.segments))
#     c= 0
#     for index, trace in enumerate(new_project.traces):
#         #print("type= %s" %(type(trace)))
#         #print("index= %s\nshape= %s\ntrace=%s\ntextin=%s\nkey=%s\n" 
#         #      %(index, trace.wave.shape, trace.wave, trace.textin, trace.key))
#         ## Appending trace and keys to dataFrame, ignore_index is necessary when 
#         ## using dictionary to append
#         #Mdf = df.append({'trace': trace.wave, 'key': trace.key}, ignore_index=True)
#         c = c + 1
#     print("len= %s" %(len(new_project.traces)))    
#     if (index != 20999):
#         print("Number of traces appended = %s,for keyVal= %s" %(index, keyVal))
#         wrongCount = wrongCount + 1
#         lessTraceList.append(keyVal)
    ## Splitting the df in train, dev and test
    ## Shuffling the data before splitting
    print("df_Size = %s before shuffling" %(len(df.index)))
    df = df.sample(frac=1).reset_index(drop=True)
    print("df_Size = %s after shuffling" %(len(df.index)))
    
    train_df = pd.concat([train_df, df.iloc[0:15000, 0:15000]])
    dev_df   = pd.concat([dev_df, df.iloc[15000:20000, 15000: 20000]])
    test_df  = pd.concat([test_df, df.iloc[20000:21000, 20000:21000]])
    
    print("len train_df = %s, dev_df = %s, test_df = %s" 
          %(len(train_df.index), len(dev_df.index), len(test_df.index)))
    ##Saving files after the batchSize is reached
    #MGif(keyVal%batchSize == batchSize-1):
    if(index%batchSize == batchSize-1):
        print("Started Saving files, count= %s, keyVal= %s" %(count, keyVal))
        ##Save file
        saveDir = "/home/manoj/Documents/chipwhisperer/jupyter/projects/captureTraces_dnn_dir/trace_key_dir/"
        saveTrain = saveDir + "/train_" + str(count) + ".pkl.zip"
        saveDev   = saveDir + "/dev_" + str(count) + ".pkl.zip"
        saveTest  = saveDir + "/test_" + str(count) + ".pkl.zip"
        
        train_df.to_pickle(saveTrain)
        dev_df.to_pickle(saveDev)
        test_df.to_pickle(saveTest)
        
        count = count + 1 ##Increment count for next saveFile
        
        print("Saved files to\n%s\n%s\n%s" %(saveTrain, saveDev, saveTest))
        ##Clear contents of the dataFrame
        clear_df(df)
        clear_df(train_df)
        clear_df(dev_df)
        clear_df(test_df)

print("Finished saving all\n")

OneFileTrace for key = 0
df_Size = 21000 before shuffling
df_Size = 21000 after shuffling
len train_df = 15000, dev_df = 5000, test_df = 1000
OneFileTrace for key = 10


/home/manoj/.local/lib/python3.6/site-packages/ipykernel_launcher.py:140: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/manoj/.local/lib/python3.6/site-packages/ipykernel_launcher.py:141: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



df_Size = 21000 before shuffling
df_Size = 21000 after shuffling
len train_df = 30000, dev_df = 10000, test_df = 2000
multiFileTrace for key = 204
prjPath= projects/captureTraces_dnn_dir//key_204.cwp
countTrace = 0

project_traces len = 3000
prjPAth = projects/captureTraces_dnn_dir//key_204.cwp
Done appending len= 3000 for key= 204, count = 0
prjPath= projects/captureTraces_dnn_dir//key_204_1.cwp
countTrace = 1

project_traces len = 3000
prjPAth = projects/captureTraces_dnn_dir//key_204_1.cwp
Done appending len= 6000 for key= 204, count = 0
prjPath= projects/captureTraces_dnn_dir//key_204_2.cwp
countTrace = 2

project_traces len = 3000
prjPAth = projects/captureTraces_dnn_dir//key_204_2.cwp
Done appending len= 9000 for key= 204, count = 0
prjPath= projects/captureTraces_dnn_dir//key_204_3.cwp
countTrace = 3

project_traces len = 12000
prjPAth = projects/captureTraces_dnn_dir//key_204_3.cwp
Done appending len= 21000 for key= 204, count = 0
prjPath= projects/captureTraces_dnn_dir//key_20

In [1]:
## Add remaining traces
import chipwhisperer as cw
import pandas as pd
import os

## dataFrame to store all the traces
df_new = pd.DataFrame(columns=['trace', 'key'])

##Adding traces for not complete 
keysNotDone = [204, 228, 244, 245]
count = 0

for index, keyVal in enumerate(keysNotDone):
    ## Setting count to zero for each new key
    count = 0 
    while count <= 5:
        if count  == 0:
            prjPath = "projects/captureTraces_dnn_dir/key_" + str(keyVal) + ".cwp"
        else:
            prjPath = "projects/captureTraces_dnn_dir/key_" + str(keyVal) + "_" + str(count) + ".cwp"

        if os.path.exists(prjPath):
            print("prjPath= %s\ncount = %s\n" %(prjPath, count))
            new_project = cw.open_project(prjPath)
        else:
            print("%s does not exist" %(prjPath))
            count = count + 1
            continue

        for index, trace in enumerate(new_project.traces):
            df_new = df_new.append({'trace': trace.wave, 'key': trace.key}, ignore_index=True)
        
        print("Done appending for key= %s, count = %s" %(keyVal, count))
        ## increment count
        count = count + 1

## Add these to the csv files.
print("Done appending all the traces\n")        

prjPath= projects/captureTraces_dnn_dir/key_204.cwp
count = 0

Done appending for key= 204, count = 0
prjPath= projects/captureTraces_dnn_dir/key_204_1.cwp
count = 1

Done appending for key= 204, count = 1
prjPath= projects/captureTraces_dnn_dir/key_204_2.cwp
count = 2

Done appending for key= 204, count = 2
prjPath= projects/captureTraces_dnn_dir/key_204_3.cwp
count = 3

Done appending for key= 204, count = 3
prjPath= projects/captureTraces_dnn_dir/key_204_4.cwp
count = 4

Done appending for key= 204, count = 4
prjPath= projects/captureTraces_dnn_dir/key_204_5.cwp
count = 5

Done appending for key= 204, count = 5
prjPath= projects/captureTraces_dnn_dir/key_228.cwp
count = 0

Done appending for key= 228, count = 0
prjPath= projects/captureTraces_dnn_dir/key_228_1.cwp
count = 1

Done appending for key= 228, count = 1
prjPath= projects/captureTraces_dnn_dir/key_228_2.cwp
count = 2

Done appending for key= 228, count = 2
prjPath= projects/captureTraces_dnn_dir/key_228_3.cwp
count = 3

Don

In [5]:
## Storing traces that were not complete to a different file

saveFile = "/home/manoj/Documents/chipwhisperer/jupyter/projects/captureTraces_dnn_dir/trace_key_dir/trace_key_204_228_244_245.csv" 
df_new.to_csv(saveFile, header=True, index=False)

In [1]:
import chipwhisperer as cw
import pandas as pd

## .to_csv is not storing all the columns in trace.
df1 = pd.DataFrame(columns=['trace', 'key'])

prjPath = "projects/captureTraces_dnn_dir/key_204.cwp"
project1 = cw.open_project(prjPath)

for index, trace in enumerate(project1.traces):
    df1 = df1.append({'trace': trace.wave, 'key': trace.key}, ignore_index=True)
df1.head()

,trace,key
0,"[0.0458984375, 0.03515625, -0.033203125, 0.019...","[204, 234, 17, 165, 196, 255, 13, 134, 58, 96,..."
1,"[0.0478515625, 0.0380859375, -0.0322265625, 0....","[204, 107, 42, 236, 121, 29, 25, 82, 67, 109, ..."
2,"[0.0458984375, 0.037109375, -0.0302734375, 0.0...","[204, 125, 28, 100, 192, 20, 180, 235, 133, 47..."
3,"[0.04296875, 0.033203125, -0.0341796875, 0.017...","[204, 2, 240, 246, 49, 17, 27, 88, 152, 126, 1..."
4,"[0.0400390625, 0.03125, -0.0302734375, 0.01367...","[204, 25, 64, 164, 12, 118, 218, 1, 70, 103, 2..."


In [48]:
## Continue with saving problem
import numpy as np

pd.set_option('display.max_colwidth', -1)
pd.reset_option('display.max_rows|display.max_columns|display.width')

trial_df = pd.DataFrame(columns={'trace', 'key'})
for index, trace in enumerate(project1.traces):
    #print("type= %s" %(type(trace.key)))
    trial_df = trial_df.append({'trace': np.array(trace.wave.copy()), 'key': trace.key}, ignore_index=True)


#print("trial_df= %s" %(trial_df.head()))
saveFile1 = "/home/manoj/Documents/chipwhisperer/jupyter/projects/captureTraces_dnn_dir/trace_key_dir/test.csv"
trial_df.to_csv(saveFile1, header=True, index=False)
print ("tracelen in df = %s, keyLen= %s" %(len(trial_df.trace[0]), len(trial_df.key[0])))
#print("df1 head\n%s" %(df1.head()))
read_df = pd.read_csv(saveFile1)
print("tracelen in read_df = %s, keyLen= %s" %(len(read_df.trace[0]), len(read_df.key[0])))
read_df.key[0]
read_df.trace[0]
read_df.head()
#print("traces= %s, len= %s" %(project1.traces, len(project1.traces)))
dir(project1.traces.__iter__)
project1.traces.__iter__

## SAving to pickle
savePickleFile = "/home/manoj/Documents/chipwhisperer/jupyter/projects/captureTraces_dnn_dir/trace_key_dir/test.pkl"
trial_df.to_pickle(savePickleFile)

df_pickle = pd.read_pickle(savePickleFile)

print ("tracelen in df_pickle = %s, keyLen pickle= %s" %(len(df_pickle.trace[0]), len(df_pickle.key[0])))
print("type trace= %s, type key = %s" %(type(df_pickle.trace[0][0]), type(df_pickle.key[0][0])))

tracelen in df = 1500, keyLen= 16
tracelen in read_df = 76, keyLen= 65
tracelen in df_pickle = 1500, keyLen pickle= 16
type trace= <class 'numpy.float64'>, type key = <class 'numpy.uint8'>


In [10]:
##Checking for panads df
print("trace= %s\nkey0= %s" %(df.iloc[3].trace, df.iloc[3].key[0]))
print("shape= %s, type= %s" %(df.iloc[3].key.shape, type(df.iloc[3].key)))
df

trace= [ 0.04785156  0.04101562 -0.02929688 ...  0.10253906  0.17089844
  0.01757812]
key0= 6
shape= (16,), type= <class 'numpy.ndarray'>


,trace,key
0,"[0.0439453125, 0.0361328125, -0.033203125, 0.0...","[6, 149, 78, 156, 53, 191, 127, 118, 78, 132, ..."
1,"[0.048828125, 0.037109375, -0.033203125, 0.023...","[6, 160, 137, 208, 158, 153, 168, 210, 50, 81,..."
2,"[0.0478515625, 0.0380859375, -0.03125, 0.02148...","[6, 194, 91, 107, 193, 182, 2, 147, 224, 49, 2..."
3,"[0.0478515625, 0.041015625, -0.029296875, 0.02...","[6, 173, 31, 31, 252, 33, 54, 1, 79, 112, 124,..."
4,"[0.0419921875, 0.0341796875, -0.03125, 0.01464...","[6, 240, 87, 14, 90, 88, 92, 113, 68, 2, 2, 20..."
5,"[0.0478515625, 0.037109375, -0.0322265625, 0.0...","[6, 104, 190, 251, 196, 128, 110, 133, 72, 54,..."
6,"[0.0478515625, 0.0380859375, -0.0322265625, 0....","[6, 96, 229, 172, 227, 129, 188, 232, 159, 113..."
7,"[0.046875, 0.0361328125, -0.033203125, 0.01953...","[6, 188, 111, 181, 27, 62, 59, 241, 14, 109, 1..."
8,"[0.04296875, 0.0341796875, -0.029296875, 0.019...","[6, 150, 233, 26, 1, 254, 87, 42, 88, 64, 250,..."
9,"[0.048828125, 0.0380859375, -0.0322265625, 0.0...","[6, 120, 150, 173, 132, 188, 20, 115, 89, 119,..."


In [4]:
import chipwhisperer as cw
ktp = cw.ktp.Basic()
key, text = ktp.next()
print("key= %s\ntext= %s" %(key, text))
print("key[0]= %s" %(key[0]))

key[0] = 15
print("new key= %s" %(key))

key= CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')
text= CWbytearray(b'7f cb e0 f8 55 49 8d 01 07 77 c6 2e f4 6d ea b3')
key[0]= 43
new key= CWbytearray(b'0f 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')


In [3]:
## Trace plot
%matplotlib inline
import matplotlib.pyplot as plt
print("preSamples= %s\nstream_mode= %s\nbasic_mode= %s" 
      %(scope.adc.presamples, scope.adc.stream_mode, scope.adc.basic_mode))

imgDir = "/home/manoj/Documents/chipwhisperer/jupyter/projects/captureTraces_dnn_dir/image/"
x=[i for i in range(scope.adc.samples)]
plotTrace = new_project.traces[0][0]

#plt.plot(x, newTrace[0], 'r', x, newTrace[1], 'g', x, newTrace[2], 'k')
plt.rcParams["legend.fontsize"] = 20
plt.rcParams["axes.labelsize"] = 30
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["font.size"] = 25

plt.figure(figsize=(60,40))
plt.plot(x, plotTrace, 'g')
plt.savefig(imgDir + "trace_0_0.png")
plt.close()

from bokeh.plotting import figure, show, save, output_file
from bokeh.io import output_notebook

output_notebook()
p = figure()
p.line(x, plotTrace, line_color='green')
show(p)
output_file(imgDir + "trace_0_0.html")
save(p)

NameError: name 'scope' is not defined

In [59]:
## Clock information
clk = scope.clock
print("adc_src= %s\nadc_freq= %s\nadc_rate= %s\nadc_locked= %s\nfreq_ctr= %s\nfreq_ctr_src= %s\nclkgen_src= %s" 
      %(clk.adc_src, clk.adc_freq, clk.adc_rate, clk.adc_locked, clk.freq_ctr, clk.freq_ctr_src, clk.clkgen_src))
print("extclk_freq= %s\nclkgen_mul= %s\nclkgen_div= %s\nclkgen_freq= %s\nclkgen_locked= %s" 
      %(clk.extclk_freq, clk.clkgen_mul, clk.clkgen_div, clk.clkgen_freq, clk.clkgen_locked))

adc_src= clkgen_x4
adc_freq= 29538459
adc_rate= 29538459.0
adc_locked= True
freq_ctr= 0
freq_ctr_src= extclk
clkgen_src= system
extclk_freq= 10000000
clkgen_mul= 2
clkgen_div= 26
clkgen_freq= 7384615.384615385
clkgen_locked= True


In [55]:
##Manoj
import os
print("CWD= %s" %(os.getcwd()))
ktp_1 = cw.ktp.Basic()
print("%s" %(dir(ktp_1)))
print("fixed_key= %s" %(ktp_1.fixed_key))
print("fixed_text= %s" %(ktp_1.fixed_text))
print("key, text tuple = %s" %(ktp_1.next(),))

##Changing fixed_key to false
ktp_1.fixed_key=False
print("key, text tuple = %s" %(ktp_1.next(),))
print("key, text tuple = %s" %(ktp_1.next(),))

CWD= /home/manoj/Documents/chipwhisperer/jupyter
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_fixedKey', '_fixedPlain', '_initPattern', '_key', '_key_len', '_name', '_text_len', '_textin', 'fixed_key', 'fixed_text', 'getInitialKey', 'getInitialText', 'getKeyType', 'getPlainType', 'get_key_type', 'init', 'initPair', 'init_pair', 'initkey', 'inittext', 'keyLen', 'key_len', 'newPair', 'new_pair', 'next', 'next_key', 'next_text', 'setInitialKey', 'setInitialText', 'setKeyType', 'setPlainType', 'setTarget', 'set_key_type', 'textLen', 'text_len', 'types', 'validateKey', 'validateText']
fixed_key= True
fixed_text= False
key, text tuple = (CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f

In [42]:
from chipwhisperer.common.utils import util
intList = [111,112,13,44,135,56,73,128]
hexList = util.list2hexstr(intList, delim=" ", prefix="0x")
print("hexList= %s" %(hexList))
byteStr = util.hexStrToByteArray(hexList)
print("byteArray= %s" %(byteStr))

##ByteArray to binary
binaryList = util.bytearray2binarylist(byteStr)
print("binaryList = %s" %(binaryList))

##hex str to list
hex2list = util.hexstr2list(hexList)
print("hex2list= %s" %(hex2list))

hexList= 0x6f 0x70 0x0d 0x2c 0x87 0x38 0x49 0x80
byteArray= CWbytearray(b'6f 70 0d 2c 87 38 49 80')
binaryList = [0 1 1 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 0 0 1 0 0 0 0
 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0]
hex2list= [111, 112, 13, 44, 135, 56, 73, 128]


In [27]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

### Analysis

In [14]:
import chipwhisperer.analyzer as cwa
leak_model = cwa.leakage_models.sbox_output
attack = cwa.cpa(project, leak_model)

cb = cwa.get_jupyter_callback(attack)   
attack_results = attack.run(cb, 100)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
PGE=,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,2B0.839,7E0.833,150.753,160.704,280.862,AE0.853,D20.787,A60.619,AB0.837,F70.800,150.759,880.722,090.509,CF0.861,4F0.818,3C0.782
1,200.244,680.210,970.203,940.175,AA0.222,9D0.207,D10.219,990.194,130.207,F40.222,AD0.192,8B0.197,B10.156,4D0.224,440.223,030.220
2,F00.204,FC0.207,1E0.203,350.168,2B0.221,AD0.200,500.214,6F0.192,A80.206,4F0.209,160.189,5B0.197,FC0.152,CC0.218,590.209,0F0.217
3,A90.200,410.204,160.200,150.168,900.202,640.198,1B0.206,240.191,940.205,D00.207,970.188,B70.181,8B0.145,530.216,860.206,A30.209
4,140.194,E20.204,CE0.197,250.168,DD0.197,2C0.195,ED0.190,2F0.163,290.193,750.197,2B0.186,670.175,800.139,770.215,710.203,370.199


In [31]:
## Manoj
print("%s\n", dir(project.traces.project))
for keys in project.traces.project.keys:
    print("key= %s" %(keys))

%s
 ['_Project__dirtyCallback', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_description', '_keys', '_name', '_segments', '_textins', '_textouts', '_traceManager', '_trace_format', '_traces', '_waves', 'addDataConfig', 'appendSegment', 'append_segment', 'checkDataConfig', 'checkDiff', 'close', 'config', 'configObjChanged', 'consolidate', 'convertDataFilepathAbs', 'createDataDirectory', 'datadirectory', 'deleteParams', 'dirty', 'export', 'filename', 'findParam', 'getClassName', 'getDataConfig', 'getDataFilepath', 'getDescription', 'getFilename', 'getName', 'getNewTraceSegment', 'getParams', 'getTraceFormat', 'get_filename', 'get_new_trace_segment', 'get_trace_format', 'hasDiffs', 

TypeError: 'IndividualIterable' object is not callable

With that, you should see red numbers fill the top of the table.

#### Plots

We'll skip most of the plots this time around. We will, however, take a look at output vs. time:

In [12]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
plot_data = cwa.analyzer_plots(attack_results)

output_notebook()
rets = []
for i in range(0, 16):
    rets.append(plot_data.output_vs_time(i))

p = figure()
for ret in rets:
    p.line(ret[0], ret[2], line_color='green')
    p.line(ret[0], ret[3], line_color='green')
    
for ret in rets:
    p.line(ret[0], ret[1], line_color='red')

show(p)

Loading BokehJS ...

Zooming in, you should see that the resulting plot looks "messier" than the 8-bit implementation:

![](https://wiki.newae.com/images/e/e3/32bit_AES_outvstime.png)

## Tests

In [13]:
key = np.array(project.keys[0])
recv_key = [kguess[0][0] for kguess in attack_results.find_maximums()]
assert np.all((key == recv_key)), "Failed to recover encryption key\nGot: {}\nExpected: {}".format(recv_key, key)

In [14]:
assert (attack_results.pge == [0]*16), "PGE for some bytes not zero: {}".format(attack_results.pge)

In [15]:
# MBED Often has a few bytes with low (0.2-0.4) correlation, causing it to fail this test
if CHECK_CORR:
    max_corrs = [kguess[0][2] for kguess in attack_results.find_maximums()]
    assert (np.all([corr > 0.6 for corr in max_corrs])), "Low correlation in attack (corr <= 0.6): {}".format(max_corrs)